## CS431/631 Big Data Infrastructure
### Fall 2021 - Assignment 1
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** Ze Sheng
* **ID:** 20848004

---
#### Overview
For this assignment, you will be using Python to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) that you used for Assignment 0.   You will also need the Python tokenizer module, `simple_tokenize.py`.

If two events $x$ and $y$ are independent, their PMI will be zero.   A positive PMI indicates that $x$ and $y$ are more likely to co-occur than they would be if they were independent.   Similarly, a negative PMI indicates that $x$ and $y$ are less likely to co-occur.   The PMI of events $x$ and $y$ is given by
\begin{equation*}
PMI(x,y) = \log\frac{p(x,y)}{p(x)p(y)}
\end{equation*}
where $p(x)$ and $p(y)$ are the probabilities of occurrence of events $x$ and $y$, and $p(x,y)$ is the probability of co-occurrence of $x$ and $y$.

For this assignment, the "events" that we are interested in are occurrences of tokens on lines of text in the input file.   For example, one event
might represent the occurence of the token "fire" a line of text, and another might represent the occurrence of the token "peace".   In that case, $p(fire)$ represents the probability that "fire" will occur on a line of text, and $p(fire,peace)$ represents the probability that *both* "fire" and "peace" will occur on the *same* line.   For the purposes of these PMI computations, it does not matter how many times a given token occures on a single line.   Either a line contains a particular token (at least once), or it does not.   For example, consider this line of text:

> three three three, said thrice

For this line, the following token-pair events have occurred:
- (three, said)
- (three, thrice)
- (said, three)
- (said, thrice)
- (thrice, three)
- (thrice, said)

Note that we are not interested in "reflexive" pairs, such as (thrice,thrice).

In addition to the probabilities of events, we will also be interested in the absolute *number* of occurences of particular events, e.g., the number of lines in which "fire" occurs.   We will use $n(x)$ to represent the these numbers.

Your main task for this assignment is to write Python code to analyze the PMI of tokens from Shakespeare's plays.    Based this analysis, we want to be able to answer two types of queries:

* Two-Token Queries: Given a pair of tokens, $x$ and $y$, report the number of lines on which that pair co-occurs ($n(x,y)$) as well as $PMI(x,y)$.
* One-Token Queries: Given a single token, $x$, report the number of lines on which that token occurs ($n(x)$).   In addition, report the five tokens that have the largest PMI with respect to $x$ (and their PMIs).   That is, report the five $y$'s for which $PMI(x,y)$ is largest.

To avoid reporting spurious results for the one-token queries, we are only interested in token pairs that co-occur a sufficient number of times.   Therefore, we will use a *threshold* parameter for one-token queries.   A one-token query should only report pairs of tokens that co-occur at least *threshold* times in the input.   For example, given the threshold 12, a one-token query for "fire" the should report the five tokens that have the largest PMI (with respect to "fire") among all tokens that co-occur with "fire" on at least 12 lines.   If there are fewer than five such tokens, report fewer than five.



Run the next block to download the text file (`Shakespeare.txt`) and the Python tokenizer module (`simple_tokenize.py`).

In [ ]:
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/Shakespeare.txt
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/simple_tokenize.py

---
#### Question 1  (2/10 marks):

Before writing code to handle the PMI queries, start writing some code to answer some simpler questions that give an
idea of how big the PMI analysis problem will be.   The box below contains some starter code that reads in the 'Shakespeare.txt' file and tokenizes it one line at time.   (This is the same code you started with for Assignment 0.)  Extend this code to determine (a) the number of *distinct* tokens that exist in 'Shakespeare.txt', and (b) the number of 
distinct token pairs that exist in 'Shakespeare.txt'.  For the purposes of this question, consider the token pair $x,y$ to be distinct from the pair $y,x$, i.e., count them both.   Ignore token pairs of the form $x,x$.

In [18]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
from itertools import permutations as perm

tokens, pairs = set(), set()
with open('Shakespeare.txt') as f:
    for line in f:
        tLine = set(simple_tokenize(line))
        # map tokens
        for t in tLine:
            tokens.add(t)
        # map pairs
        for p in perm(tLine, 2):
            pairs.add(p)
        

# extend this code to answer Question 1.
# when your code is executed, it should print the number of distinct tokens and the number of distinct token pairs
print(len(tokens))
print(len(pairs))

25975
1969760


---

#### Question 2 (6/10 marks):
Next, write Python code to answer the one-token and two-token queries described above, for 'Shakespeare.txt'.   The code cell below contains some starter code that implements a simple text-based query interface.  It allows a user to ask a series of one-token or two-token queries.   Try running the starter code to get a sense of how the interface behaves.    

Your task is to write code to read and tokenize 'Shakespeare.txt', record information that will allow both types of PMI queries to be answered, and then answer queries that are posed through the query interface.  Make sure that your code is well commented, so that it will be clear to the markers.

If you cannot get answers to both types of queries working, try to get at least one type working, for partial credit.


In [37]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
# the log function for computing PMI
# for the sake of consistency across solutions, please use log base 10
from math import log
from collections import defaultdict, Counter

###################################################################################################################
#  replace this with your PMI analysis code, so that you can support the user interface below
#  it should read and tokenize Shakespeare.txt, and store enough information in Python data structures
#  to allow you to answer the PMI queries below
###################################################################################################################
class Analyzer():
    def __init__(self, threshold=0):
        self.fileName = 'Shakespeare.txt'
        self.threshold = threshold
        self.cntLine = 0
        self.db = {}
        self._generate()
    
    def _get_stripe(self, token):
        stripe = self.db.get(token)
        if stripe:
            return stripe
        else:
            # initialize a defaultdict for the new acquaintance
            stripe = defaultdict(int)
            stripe[token] = 0
            self.db[token] = stripe
            return stripe

    def _generate(self):
        with open(self.fileName) as f:
            # traverse the lines
            # O(L * est(n)^2), where L is the number of lines, 
            # est(n) is the estimation of the average number of unique tokens in each line
            for line in f:
                self.cntLine += 1
                tLine = set(simple_tokenize(line))
                # count occurence and co-occurence
                # O(n^2), where n = len(tLine)
                for t in tLine:
                    stripe = self._get_stripe(t)
                    for coT in tLine:
                        stripe[coT] += 1

    def _PMI(self, cooN, occNx, occNy):
        return log(cooN * self.cntLine / (occNx * occNy), 10)
    
    def set_threshold(self, threshold):
        self.threshold = threshold
    
    def oneTok_query(self, token):
        stripe = self._get_stripe(token)
        # calculate PMI of the valid pairs
        heap = Counter()
        for coT, occNy in Counter(stripe).most_common()[1:]:
            if occNy >= self.threshold:
                cooN, pmi = self.twoTok_query(token, coT)
                heap[(coT, cooN)] = pmi
            else:
                break
        # fetch the top-5 results
        ret = []
        for (coT, cooN), pmi in heap.most_common(5):
            ret.append((coT, cooN, pmi))
        return stripe[token], ret

    def twoTok_query(self, x, y):
        stripe = self.db[x]
        cooN, occNx, occNy = stripe[y], stripe[x], self.db[y][y]
        return cooN, self._PMI(cooN, occNx, occNy)


###################################################################################################################
#  the user interface below defines the types of PMI queries that users can ask
#  you will need to modify it - where indicated - to access the results of your PMI analysis (above)
#  so that the queries can be answered
###################################################################################################################

# initialize the task here
task = Analyzer()
while True:
    q = input("Input 1 or 2 space-separated tokens (return to quit): ")
    if len(q) == 0:
        break
    q_tokens = simple_tokenize(q)
    if len(q_tokens) == 1:
        threshold = 0
        while threshold <= 0:
            try:
                threshold = int(input("Input a positive integer frequency threshold: "))
            except ValueError:
                print("Threshold must be a positive integer!")
                continue
        
        # Put code here to answer a One-Token Query with token q_tokens[0] and the specified threshold,
        # and output the result.
        # The print() statements below exist to show you the desired output format.
        # Replace them with your own output code, which should produce results in a similar format.

        task.set_threshold(threshold)
        occN, ret = task.oneTok_query(q_tokens[0])

        print("  n({0}) = {1}".format(q_tokens[0], occN))   # still print out this line if occN < threshold
        print("  high PMI tokens with respect to {0} (threshold: {1}):".format(q_tokens[0], threshold))
        for coT, cooN, pmi in ret:
            print("    n({0},{1}) = {2},  PMI({0},{1}) = {3}".format(q_tokens[0], coT, cooN, pmi))

    elif len(q_tokens) == 2:
        # Put code here to answer a Two-Token Query with tokens q_tokens[0] and q_tokens[1]
        # As was the case for the One-Token query, the print statements below show the desired output format
        # Replace them with your own output code

        cooN, pmi = task.twoTok_query(q_tokens[0], q_tokens[1])

        print("  n({0},{1}) = {2}".format(q_tokens[0], q_tokens[1], cooN))
        print("  PMI({0},{1}) = {2}".format(q_tokens[0], q_tokens[1], pmi))
        
    else:
        print("Input must consist of 1 or 2 space-separated tokens!")


  n(he,she) = 42
  PMI(he,she) = -0.36577689462006124
  n(she,he) = 42
  PMI(she,he) = -0.36577689462006124
  n(of) = 16624
  high PMI tokens with respect to of (threshold: 12):
    n(of,syracuse) = 228,  PMI(of,syracuse) = 0.8596984941000634
    n(of,ephesus) = 155,  PMI(of,ephesus) = 0.8427394821131029
    n(of,instead) = 15,  PMI(of,instead) = 0.8392229083902657
    n(of,antipholus) = 201,  PMI(of,antipholus) = 0.8319911958063934
    n(of,brace) = 14,  PMI(of,brace) = 0.8092596850128225
  n(the) = 24300
  high PMI tokens with respect to the (threshold: 12):
    n(the,proudest) = 14,  PMI(the,proudest) = 0.7023808886052992
    n(the,duke's) = 54,  PMI(the,duke's) = 0.6865866214220675
    n(the,count's) = 14,  PMI(the,count's) = 0.6724176652278561
    n(the,bridge) = 13,  PMI(the,bridge) = 0.670196205233898
    n(the,commons) = 30,  PMI(the,commons) = 0.6609882034470743
  n(the) = 24300
  high PMI tokens with respect to the (threshold: 100):
    n(the,king's) = 146,  PMI(the,king's) =

---

#### Question 3 (2/10 marks):

Suppose that you try to run your PMI analysis on larger files:  say, 10 times, or 100 times, or 1000 times larger than 'Shakespeare.txt'.    As the input file grows larger, what will happen to the execution of your program?   Will it get slower?   How much slower?   Will it eventually fail to run?   If so, why?

In the cell below, briefly (one or two paragraphs), discuss what will happen if the input to your analysis grows.  We're not looking for an exact or empirical analysis of the behaviour of your program as a function of input size.  Rather, we are looking for a discussion of trends and limits.

#### Answer to Question 3:

Assume that,

`L` denotes the number of lines, 

`n` denotes the number of unique tokens in a line, 

`est(n)` denotes the estimation of the average number of unique tokens in each line.

`s` denotes the size of the input file.


We use the function `_generate(self)` to count and cache the occurrences. FYI, the related code sections are commented. 

The common-case time-complexity of this step (before queries) is `O( L * est(n)^2 )`. As a reasonable assumption, 'est(n)' is a invariable mapping of `s`, which means `est(n)` keeps stable while the input file is becoming larger. Then, `L` is a linear mapping of `s`. As a result, the overall time-complexity can be considered as `O(s)`, where `s` is the size of the input file, i.e., **it grows linearly**. 

As for queries, this step takes `O( s * log(s) )` time in the worst-case. 

Eventually, **it will fail** to run. The reason is that we use dictionary to save the intermediate results in RAM. It might overflow if the input tokens are too many. 


---
Don't forget to save your workbook!   (It's a good idea to do this regularly, while you are working.)   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.